# 02 – Model Training

In this notebook, we train the MLPClassifier model using the PyTorch training loop defined in `train.py`.

We will:
- Load preprocessed data
- Create datasets and dataloaders
- Instantiate the model
- Define loss function and optimizer
- Train the model for 20 epochs
- Save the best model based on validation accuracy


In [19]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from src.model import MLPClassifier
from src.train import train_model
from src.data_loader import HumanActivityDataset
import pickle
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Device:", device)


✅ Device: cpu


## Load processed data

We load the preprocessed NumPy arrays saved at the end of the `01_exploration.ipynb` notebook.


In [20]:
with open("../data/processed_data.pkl", "rb") as f:
    X_train_final, y_train_final, X_val, y_val, X_test_scaled, y_test_encoded = pickle.load(f)


In [21]:
BATCH_SIZE = 64

train_dataset = HumanActivityDataset(X_train_final, y_train_final)
val_dataset = HumanActivityDataset(X_val, y_val)
test_dataset = HumanActivityDataset(X_test_scaled, y_test_encoded)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


## Define MLP architecture

We will use:
- 561 input features
- Two hidden layers: 256 and 128 neurons
- ReLU activation
- Dropout (0.3)
- Output layer with 6 classes


In [22]:
model = MLPClassifier(
    input_dim=561,
    hidden_dims=[256, 128],
    output_dim=6,
    dropout_rate=0.3
).to(device)

print(model)


MLPClassifier(
  (model): Sequential(
    (0): Linear(in_features=561, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=256, out_features=128, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=128, out_features=6, bias=True)
  )
)


## Define loss function and optimizer

We use:
- CrossEntropyLoss (for multi-class classification)
- Adam optimizer with learning rate = 0.001


In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


## Train the model

We will train for 20 epochs and save the best model based on validation accuracy.


In [27]:
history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,         # ✅ esta ordem é importante!
    criterion=criterion,
    device=device,
    epochs=20,
    save_path="../outputs/best_model.pth"
)



Epoch 1/20


TypeError: 'Adam' object is not callable